In [1]:
""" reference:
        https://qiita.com/usaito/items/09daccdd91bc98c21dff 
"""

' reference:\n        https://qiita.com/usaito/items/09daccdd91bc98c21dff \n'

In [3]:
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import roc_curve, auc

# Case Study 1
### Right Heart Catheterization Dataset (RHC Dataset)

In [2]:
# read data
data_df = pd.read_csv("http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/rhc.csv")
data_df.head()

,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,...,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,1,COPD,NaN,Yes,11142,11151.0,NaN,11382,No,0,...,No,No,No,No,No,0.0,NaN,white,Under $11k,5
1,2,MOSF w/Sepsis,NaN,No,11799,11844.0,11844.0,11844,Yes,1,...,No,No,Yes,No,No,NaN,1437.0,white,Under $11k,7
2,3,MOSF w/Malignancy,MOSF w/Sepsis,Yes,12083,12143.0,NaN,12400,No,0,...,No,No,No,No,No,NaN,599.0,white,$25-$50k,9
3,4,ARF,NaN,No,11146,11183.0,11183.0,11182,Yes,0,...,No,No,No,No,No,NaN,NaN,white,$11-$25k,10
4,5,MOSF w/Sepsis,NaN,No,12035,12037.0,12037.0,12036,Yes,0,...,No,No,No,No,No,NaN,64.0,white,Under $11k,11


In [3]:
# the cross-tabulation for the motality rate and RHC
pd.crosstab(data_df.death, data_df.swang1)

swang1,No RHC,RHC
death,,
No,1315,698
Yes,2236,1486


In [4]:
# calculate the difference of the motality rate between RHC and No RHC
rhc = 1486 / (698 + 1486)
no = 2236 / (1315 + 2236)
rhc - no

0.050721150622586864

## Estimate the propensity score using statsmodels.api.Logit

In [5]:
# select independent variables
cols = ["cat1", "sex", "race", "edu", "income",
        "resp", "card", "neuro", "gastr", "renal", "meta", "hema", "seps", "trauma", "ortho",
        "das2d3pc", "dnr1", "ca", "surv2md1", "aps1", "scoma1", "wtkilo1", "temp1",
        "resp1", "hrt1", "pafi1", "paco21", "ph1", "wblc1", "hema1", "sod1", "pot1", "crea1",
        "bili1", "alb1", "cardiohx", "chfhx", "dementhx", "psychhx", "chrpulhx", "renalhx",
        "liverhx", "gibledhx", "immunhx", "transhx", "amihx",
        "age", "meanbp1"]

# categorical independent variables
categorical_columns = ["cat1", "sex", "race", "edu", "income", "ca", "dnr1",
                       "resp", "card", "neuro", "gastr", "renal", "meta", "hema", "seps", "trauma", "ortho"]

# make dummies for catecorical independent variables
X = data_df[cols]
dummy = pd.get_dummies(X[categorical_columns], drop_first=True)
X = pd.concat([X, dummy], axis=1)
X = X.drop(categorical_columns, axis=1)

In [6]:
# add an intercept
X.loc[:, "Intercept"] = 1

In [7]:
# get a dummy for RHC or not
z1 = pd.get_dummies(data_df["swang1"])["RHC"]

In [8]:
# the dependent variable
y = pd.get_dummies(data_df["death"])["Yes"]

In [9]:
glm = sm.Logit(z1, X)
result = glm.fit()
ps = result.predict(X)

Optimization terminated successfully.
         Current function value: inf
         Iterations 6


C:\Users\rhoriuchi\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1724: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


In [11]:
# Calculate AUC in terms of C-statistic
fpr, tpr, thresholds = roc_curve(z1, ps)
auc(fpr, tpr) # C-statistic is to be desired greater than 0.80

0.7963133740379587

## Estimate ATE by applying IPW

In [12]:
# Average Treatment Effect
ipwe1 = sum((z1*y)/ps) / sum(z1/ps)
ipwe0 = sum((1-z1)*y/(1-ps)) / sum((1-z1)/(1-ps))
ATE = ipwe1 - ipwe0
print(ATE)

0.05907512789883529


In [ ]:
"""
Remember that
    # calculate the difference of the motality rate between RHC and No RHC
    rhc = 1486 / (698 + 1486)
    no = 2236 / (1315 + 2236)
    rhc - no
returns 0.050721150622586864, the estimated ATE grows by 0.009.
"""

# Case Study 2
### the utilization rate of the smart phone app

In [4]:
# read data
data_df = pd.read_csv('https://github.com/iwanami-datascience/vol3/raw/master/kato%26hoshino/q_data_x.csv')
data_df.head()

,cm_dummy,gamedummy,area_kanto,area_keihan,area_tokai,area_keihanshin,age,sex,marry_dummy,job_dummy1,...,T,F1,F2,F3,M1,M2,M3,TVwatch_day,gamesecond,gamecount
0,0,0,0,0,0,1,44.5,1,1,1,...,0,0,0,0,0,1,0,33.427600,0,0
1,0,0,0,1,0,0,34.5,1,1,1,...,0,0,0,0,0,1,0,31.542862,0,0
2,0,0,0,1,0,0,24.5,1,0,0,...,0,0,0,0,1,0,0,37.825805,0,0
3,0,0,0,1,0,0,44.5,1,1,1,...,0,0,0,0,0,1,0,36.345911,0,0
4,0,0,0,1,0,0,34.5,1,1,1,...,0,0,0,0,1,0,0,49.344942,0,0


In [5]:
# cross tabulation
pd.crosstab(data_df["gamedummy"], data_df["cm_dummy"])

cm_dummy,0,1
gamedummy,,
0,5428,3832
1,428,312


In [6]:
# the difference of the utilization rate between having watched CM or not
312/(3832+312) - 428/(5428+428)

0.002202143595586223

In [7]:
# the difference of the utilization count between having watched CM or not
cm1 = data_df[data_df.cm_dummy == 1].gamecount.mean()
cm0 = data_df[data_df.cm_dummy == 0].gamecount.mean()
cm1 - cm0

-1.4845493913116865

In [8]:
# the difference of the utilization time between having watched CM or not
cm1 = data_df[data_df.cm_dummy == 1].gamesecond.mean()
cm0 = data_df[data_df.cm_dummy == 0].gamesecond.mean()
cm1 - cm0

-629.6405765396544

## Estimate the propensity score using statsmodels.api.Logit

In [9]:
# define independent variables
cols = ["age", "sex", "TVwatch_day", "marry_dummy", "child_dummy", "inc", "pmoney",
        "area_kanto", "area_tokai", "area_keihanshin",
        "job_dummy1", "job_dummy2", "job_dummy3", "job_dummy4", "job_dummy5", "job_dummy6",
        "fam_str_dummy1", "fam_str_dummy2", "fam_str_dummy3", "fam_str_dummy4"]

X = data_df[cols].copy()

In [10]:
# add the intercept
X.loc[:, "Intercept"] = 1

In [11]:
# get the dummy of having watched CM or not
z1 = data_df.cm_dummy

In [12]:
# get dependent variables (1: the utilization rate, 2: _ counts, 3: _ time)
y1 = data_df.gamedummy
y2 = data_df.gamecount
y3 = data_df.gamesecond

In [13]:
# estimate the propensity score by using statsmodels.api.Logit
glm = sm.Logit(z1, X)
result = glm.fit()
ps = result.predict(X)
print(ps)

Optimization terminated successfully.
         Current function value: 0.542152
         Iterations 6
0       0.046217
1       0.255983
2       0.177427
3       0.227593
4       0.242373
5       0.158504
6       0.048143
7       0.124441
8       0.228190
9       0.259171
10      0.243793
11      0.325195
12      0.058288
13      0.287807
14      0.058983
15      0.029108
16      0.227459
17      0.237748
18      0.303732
19      0.039849
20      0.268014
21      0.161931
22      0.305485
23      0.301188
24      0.429793
25      0.368691
26      0.268220
27      0.457681
28      0.268657
29      0.433015
          ...   
9970    0.410883
9971    0.410965
9972    0.410936
9973    0.410955
9974    0.410962
9975    0.410923
9976    0.410920
9977    0.248503
9978    0.248498
9979    0.248479
9980    0.248485
9981    0.248488
9982    0.248487
9983    0.248477
9984    0.273182
9985    0.273148
9986    0.273143
9987    0.273199
9988    0.273158
9989    0.273165
9990    0.273139
9991    0.2731

In [14]:
# calculate AUC in terms of C-statistic
fpr, tpr, thresholds = roc_curve(z1, ps)
auc(fpr, tpr)

0.7917012811992321

## Estimate ATE by applying IPW

In [15]:
def ate_calc(y, z, e):
    """ 
    arguments:
        y: the dependent variable
        z: the treatment allocation variable
        e: the estimated propensity score
    return:
        ate: the estimated average treatment effect
    """
    ipwe1 = sum((z*y)/e) / sum(z/e) # Treated
    ipwe0 = sum(((1-z)*y)/(1-e)) / sum((1-z)/(1-e)) # Control
    ate = ipwe1 - ipwe0
    return ate

In [17]:
# 1: Estimate the ATE w.r.t. the utilization rate
ate1 = ate_calc(y1, z1, ps)
print("the ATE w.r.t. the utilization rate: ", ate1)

# 2: Estimate the ATE w.r.t. the utilization counts
ate2 = ate_calc(y2, z1, ps)
print("the ATE w.r.t. the utilization counts: ", ate2)

# 3: Estimate the ATE w.r.t. the utilization time
ate3 = ate_calc(y3, z1, ps)
print("the ATE w.r.t. the utilization time: ", ate3)

the ATE w.r.t. the utilization rate:  0.03231177330512078
the ATE w.r.t. the utilization counts:  5.349029566474563
the ATE w.r.t. the utilization time:  1513.6996907825041
